In [2]:
import time
from binance.client import Client
import psycopg2
from psycopg2 import sql
import pandas as pd
import threading

# Initialize the Binance API client
api_key = 'vijG9ARQVjevGky2bkTgFACUMmRJBVxVqwtzDIONOToNIw4YmTMxMgyy4PEVUhFH'
api_secret = 'f9BHEpkASNW9S0nd7xe1Iq55ccd0bFrwI8LL0mA0UlIgODhubsX8noG6TnWicvmh'
client = Client(api_key, api_secret)

# List of coins to fetch data for
coins = ['sxp', 'chess', 'blz', 'joe', 'perl']

def get_db_connection():
    connection = psycopg2.connect(
        host='ahsanpost.postgres.database.azure.com',
        user='ahsan',
        password='name@123',
        dbname='postdb',
        sslmode='require'
    )
    return connection

# Create tables for each coin
def create_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    for coin in coins:
        cursor.execute(sql.SQL("""
            CREATE TABLE IF NOT EXISTS {} (
                id SERIAL PRIMARY KEY,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                price NUMERIC(18, 8),
                volume NUMERIC(18, 8)
            )
        """).format(sql.Identifier(f'{coin}usdt')))
    connection.commit()
    connection.close()

def fetch_and_store_recent_trades(coin):
    last_trade_timestamp = None
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            trades = client.get_recent_trades(symbol=f'{coin.upper()}USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['time'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(sql.SQL("INSERT INTO {} (timestamp, price, volume) VALUES (%s, %s, %s)").format(sql.Identifier(f'{coin}usdt')), (trade_timestamp, trade['price'], trade['qty']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin}: {e}')
            connection.rollback()
            # Wait for 60 seconds before retrying
            time.sleep(60)
            continue
        finally:
            connection.close()
        time.sleep(10)


# Function to delete data older than 1 hour
def delete_old_data(coin):
    while True:
        connection = get_db_connection()
        cursor = connection.cursor()
        try:
            cursor.execute(sql.SQL("DELETE FROM {} WHERE timestamp < NOW() - INTERVAL '1 HOUR'").format(sql.Identifier(f'{coin}usdt')))
            connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            connection.rollback()
        finally:
            connection.close()
        time.sleep(300)  # Sleep for 5 minutes

# Start the threads
def start_threads():
    create_tables()
    for coin in coins:
        threading.Thread(target=fetch_and_store_recent_trades, args=(coin,)).start()
        threading.Thread(target=delete_old_data, args=(coin,)).start()

start_threads()

while True:
    time.sleep(2)


OperationalError: (2003, "Can't connect to MySQL server on 'ahsanpost.postgres.database.azure.com' (timed out)")